In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# !pip install import-ipynb
# !pip install albumentations==0.4.6

In [ ]:
# !cp '/content/drive/MyDrive/Colab Notebooks/Covid19/Phase2_IncorporatingBinaryClassification/Covid19_Phase2_HUUtils.ipynb' .
# !cp '/content/drive/MyDrive/Colab Notebooks/Covid19/Covid19_CustomCrAsppReUNet.ipynb' .

In [ ]:
%cd ~/Minh\ Hieu

In [ ]:
# saving_directory = "/content/drive/MyDrive/Colab Notebooks/Covid19/Phase2_IncorporatingBinaryClassification/Output/LamdaNetCustom"

# saving_directory = "Output/CRNetPNG_NOLung_314E"
# saving_directory = "Output/CRNetPNG_NOLung_314E512"
# saving_directory = "Output/CRNetPNG_YESLung_314E"

# saving_directory = "Output/CRNetPNG_YESLung_314E_CAP"

# saving_directory = "Output/CRNetHU_NOLung_314E"
# saving_directory = "Output/CRNetHU_YESLung_314E512"

# saving_directory = "Output/CutomCrAsppReUNetHU_YESLung_108E"
# saving_directory = "Output/CutomCrAsppReUNetPNG_YESLung_108E"

# saving_directory = "Output/CutomCrAsppReUNetPNG_YESLung_108E_CAP"
# saving_directory = "Output/CutomCrAsppReUNetPNG_YESLung_108E_CAP"

# saving_directory = "Output/OriginCrAsppReUNetHU_YESLung_108E"
# saving_directory = "Output/OriginCrAsppReUNetPNG_YESLung_108E"

# epochs = 50
# epochs = 108
# epochs = 314
#######################################################################################

# saving_directory = "Final_Output/XNet_PNG_256"                      ## 450   
# saving_directory = "Final_Output/OriginalIncoNet_PNG_256"           ## 450    
# saving_directory = "Final_Output/XNet_HU_256"                       ## 450
# saving_directory = "Final_Output/OriginalIncoNet_HU_256"            ## 450     

# saving_directory = "Final_Output/XNet_CAP_HU_256"                   ## 450   
# saving_directory = "Final_Output/XNet_CAP_PNG_256"                  ## 450   

# saving_directory = "Final_Output/CRNet_HU_256"                      ## 450
# saving_directory = "Final_Output/CRNet_PNG_256"                     ## 450 

saving_directory = "Final_Output/XNet_512"                          ## 450 


epochs = 450
start_epoch = 0         #if resume training, change to the last training index

is_load_to_ram = False  #load training data to RAM
is_continue = False     #resume trainning

is_HU = True            #HU or 255 (DICOM or PNG)
HU_range = 900.0        #from -1000 to -100
is_concat_lung = True   #keep as True (False for NO concatenation of lung masks)
is_full_size = True     #512 or 256
is_cap = False          #classification with CAP or healthy cases

# #uncomment to select model 
# model_name = "crnet"                  #CRNet
# model_name = "crasppreunet"           #CR-IM
model_name = "crasppreunet_custom"    #CR-IM-SCRC

# model_name = "densenet"
# is_pretrained = True

In [ ]:
import albumentations as A
import import_ipynb
import torch
import cv2
import torch.nn as nn
import torch.optim as optim
import numpy as np
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from Covid19_Phase2_Utils_EvalTest import evaluate_model_incorporate, evaluate_model_single, save_checkpoint, load_checkpoint, log_epoch, log_train
from Covid19_Phase2_Utils_Dataset import get_png_data_loader, get_hu_data_loader
from Covid19_Phase2_Utils_Train import DiceBCEWithLogisticLoss, CustomAccuracyLoss, FocalTverskyLoss, train_single, train_incorporate
from Covid19_CRNet import CRNet
from Covid19_CrAsppReUNet import CRASPPReUNet
# from Covid19_CustomCrAsppReUNet import CustomCRASPPReUNet
from Covid19_CustomFinalCrAsppReUNet import CustomFinalCRASPPReUNet
import torchvision.models as models

In [ ]:
def load_densenet(pretrained=False):
  print(f"Model Pretrained is {pretrained}")
  model = models.densenet169(pretrained=pretrained)
  num_feature = model.classifier.in_features
  model.classifier = nn.Linear(num_feature, 2)
  return model.cuda(), "single"  

In [ ]:
def load_crnet():
  model = CRNet(img_dimwh=height_crop, in_channels=in_channels)
  return model.cuda(), "single"

In [ ]:
def load_crasppreunet():
  model = CRASPPReUNet(img_dimwh=height_crop, in_channels=in_channels)
  return model.cuda(), "incorporate"

In [ ]:
def load_crasppreunet_custom():
  model = CustomFinalCRASPPReUNet(img_dimwh=height_crop, in_channels=in_channels)
  return model.cuda(), "incorporate"

In [ ]:
def init_weights(module):
  if type(module) == nn.Linear or\
    type(module) == nn.Conv2d or\
    type(module) == nn.ConvTranspose2d:
    torch.nn.init.kaiming_normal_(module.weight)
  elif type(module) == nn.BatchNorm2d:
    nn.init.ones_(module.weight)
    nn.init.zeros_(module.bias)

In [ ]:
checkpoint_path = saving_directory + "/checkpoint.pth.tar"
train_log_filename = saving_directory + "/train_log"

if not is_HU:
  covid_dir = 'Dataset_PNG/COVID'
  nonCovid_dir = 'Dataset_PNG/NONCOVID'
else:
  covid_dir = 'zip/Dataset/COVID'
  nonCovid_dir = 'zip/Dataset/NONCOVID'

if not is_full_size:
  height = width = 256
  height_crop = width_crop= 224
  blur_kernel = 5
else:
  height = width = 512
  height_crop = width_crop= 448
  blur_kernel = 7

rotation_limit = 15
mean = [0.0, 0.0, 0.0]
std = [1.0, 1.0, 1.0]
random_crop_scale = 0.8
max_pixel_value = 255.0
contrast_factor = brightness_factor = 0.2

batch_size = 16
# batch_size = 64
learning_rate_class = 1e-4
learning_rate_mask = 1e-5
reduction = 'mean'
# reduction = 'sum'
scheduler_period_class = 10
scheduler_period_mask = 10
if is_concat_lung:
  in_channels = 4
else:
  in_channels = 3

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("[INFO] Using " + device + " for training ...")

if not is_HU:
  if not is_cap:
    train_list_COVID = 'Dataset_PNG/covid_train.txt'
    train_list_NonCOVID = 'Dataset_PNG/normal_train.txt'

    val_list_COVID = "Dataset_PNG/covid_validation.txt"
    val_list_NonCOVID = "Dataset_PNG/normal_validation.txt"

    test_list_COVID = "Dataset_PNG/covid_test.txt"
    test_list_NonCOVID = "Dataset_PNG/normal_test.txt"
  else:
    print("[INFO] CAP")
    train_list_COVID = 'Dataset_PNG/covid_train.txt'
    train_list_NonCOVID = 'Dataset_PNG/cap_train.txt'

    val_list_COVID = "Dataset_PNG/covid_validation.txt"
    val_list_NonCOVID = "Dataset_PNG/cap_validation.txt"

    test_list_COVID = "Dataset_PNG/covid_test.txt"
    test_list_NonCOVID = "Dataset_PNG/cap_test.txt"
else:
  if not is_cap:
    # train_list_COVID = 'zip/Dataset/covid_train (copy).txt'       #small set for testing convergence
    # train_list_NonCOVID = 'zip/Dataset/normal_train (copy).txt'   #small set for testing convergence
    train_list_COVID = 'zip/Dataset/covid_train.txt'
    train_list_NonCOVID = 'zip/Dataset/normal_train.txt'

    val_list_COVID = "zip/Dataset/covid_validation.txt"
    val_list_NonCOVID = "zip/Dataset/normal_validation.txt"

    test_list_COVID = "zip/Dataset/covid_test.txt"
    test_list_NonCOVID = "zip/Dataset/normal_test.txt"
  else:
    print("[INFO] CAP")   
    train_list_COVID = 'zip/Dataset/covid_train.txt'
    train_list_NonCOVID = 'zip/Dataset/cap_train.txt'

    val_list_COVID = "zip/Dataset/covid_validation.txt"
    val_list_NonCOVID = "zip/Dataset/cap_validation.txt"

    test_list_COVID = "zip/Dataset/covid_test.txt"
    test_list_NonCOVID = "zip/Dataset/cap_test.txt"

[INFO] Using cuda:0 for training ...


In [ ]:
if not is_HU:
  train_transformer = A.Compose([
    A.Resize(height=height, width=width, interpolation=cv2.INTER_AREA),
    A.RandomResizedCrop(height=height_crop, width=width_crop, scale=(random_crop_scale, 1.0), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(),
    A.Rotate(limit=rotation_limit),
    A.GaussNoise(),
    A.GaussianBlur(blur_limit=blur_kernel),
    A.RandomContrast(limit=contrast_factor),
    A.RandomBrightness(limit=brightness_factor),
    A.Normalize(
        mean=mean,
        std=std,
        max_pixel_value=max_pixel_value,
    ),
    ToTensorV2()
    ],
    additional_targets={'lung_mask': 'mask', 'lesion_mask': 'mask'},
    )
  
  val_transformer = A.Compose([
  A.Resize(height=height_crop, width=width_crop),
  A.CenterCrop(height=height_crop, width=width_crop),
  A.Normalize(
      mean=mean,
      std=std,
      max_pixel_value=max_pixel_value,
  ),
  ToTensorV2()
  ],
  additional_targets={'lung_mask': 'mask', 'lesion_mask': 'mask'},
  )
else:
  train_transformer = A.Compose([
    A.ToFloat(max_value=HU_range),
    A.Resize(height=height, width=width, interpolation=cv2.INTER_AREA),
    A.RandomResizedCrop(height=height_crop, width=width_crop, scale=(random_crop_scale, 1.0), interpolation=cv2.INTER_AREA),
    A.HorizontalFlip(),
    A.Rotate(limit=rotation_limit),
    A.GaussianBlur(blur_limit=blur_kernel),
    A.RandomContrast(limit=contrast_factor),
    A.RandomBrightness(limit=brightness_factor),
    ToTensorV2()
    ],
    additional_targets={'lung_mask': 'mask', 'lesion_mask': 'mask'},
    )

  val_transformer = A.Compose([
    A.ToFloat(max_value=HU_range),
    A.Resize(height=height_crop, width=width_crop),
    A.CenterCrop(height=height_crop, width=width_crop),
    ToTensorV2()
    ],
    additional_targets={'lung_mask': 'mask', 'lesion_mask': 'mask'},
    )

In [ ]:
if model_name == "densenet":
  model, mode = load_densenet(pretrained=is_pretrained)
if model_name == "crnet":
  model, mode = load_crnet()
if model_name == "crasppreunet":
  model, mode = load_crasppreunet()
if model_name == "crasppreunet_custom":
  model, mode = load_crasppreunet_custom()

In [ ]:
model.apply(init_weights)

In [ ]:
# loss_fn_class = nn.BCEWithLogitsLoss(reduction=reduction).cuda()
loss_fn_class = DiceBCEWithLogisticLoss(reduction=reduction).cuda()
# loss_fn_mask = nn.BCEWithLogitsLoss(reduction=reduction).cuda()
# loss_fn_mask = DiceBCEWithLogisticLoss(reduction=reduction).cuda()
# loss_fn_mask = CustomAccuracyLoss().cuda()
loss_fn_mask = FocalTverskyLoss().cuda()
optimizer_class = optim.Adam(model.parameters(), lr=learning_rate_class)
optimizer_mask = optim.Adam(model.parameters(), lr=learning_rate_mask)
scheduler_class = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer_class, T_max=scheduler_period_class)
scheduler_mask = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer_mask, T_max=scheduler_period_mask)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
if not is_HU:
  train_loader, val_loader, test_loader = get_png_data_loader(
      covid_dir=covid_dir,
      nonCovid_dir=nonCovid_dir,
      train_list_COVID=train_list_COVID,
      train_list_NonCOVID=train_list_NonCOVID,
      train_transformer=train_transformer,
      val_list_COVID=val_list_COVID,
      val_list_NonCOVID=val_list_NonCOVID,
      val_transformer=val_transformer,
      test_list_COVID=test_list_COVID,
      test_list_NonCOVID=test_list_NonCOVID,
      batch_size=batch_size,
      is_load_to_ram=is_load_to_ram,
      is_concat_lung=is_concat_lung,
      is_cap=is_cap
  )
else:
  train_loader, val_loader, test_loader = get_hu_data_loader(
      covid_dir=covid_dir,
      nonCovid_dir=nonCovid_dir,
      train_list_COVID=train_list_COVID,
      train_list_NonCOVID=train_list_NonCOVID,
      train_transformer=train_transformer,
      val_list_COVID=val_list_COVID,
      val_list_NonCOVID=val_list_NonCOVID,
      val_transformer=val_transformer,
      test_list_COVID=test_list_COVID,
      test_list_NonCOVID=test_list_NonCOVID,
      batch_size=batch_size,
      is_load_to_ram=is_load_to_ram,
      is_concat_lung=is_concat_lung,
      is_cap=is_cap
  )

In [ ]:
start = 0
if is_continue:
  load_checkpoint(torch.load(checkpoint_path.replace(".pth.tar", str(start_epoch - 1) + ".pth.tar")), model, optimizer_class, optimizer_class)
  start = start_epoch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# train
for epoch in range(start, epochs):
  print("\nEpoch:", epoch)
  if mode == "incorporate":
    loss_class, loss_mask, accuracy_class, accuracy_mask, dice_class, dice_mask = train_incorporate(
        loader=train_loader, 
        model=model,
        optimizer_class=optimizer_class,
        optimizer_mask=optimizer_mask, 
        loss_fn_class=loss_fn_class,
        loss_fn_mask=loss_fn_mask, 
        scaler=scaler, 
        device=device)
  if mode == "single":
    loss_class, loss_mask, accuracy_class, accuracy_mask, dice_class, dice_mask = train_single(
        loader=train_loader, 
        model=model,
        optimizer=optimizer_class,
        loss_fn=loss_fn_class,
        scaler=scaler, 
        device=device)
  log_train(epoch, loss_class, loss_mask, accuracy_class, accuracy_mask, dice_class, dice_mask, train_log_filename)
  
  if mode == "incorporate":
    f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_incorporate(loader=val_loader, model=model, device=device)
  if mode == "single":
    f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_single(loader=val_loader, model=model, device=device)

  log_epoch("EVAL", epoch, accuracy_class, f1, dice_class, accuracy_mask, dice_mask, train_log_filename)
  checkpoint = {
      "epoch": epoch,
      "state_dict": model.state_dict(),
      "optimizer_class": optimizer_class.state_dict(),
      "optimizer_mask": optimizer_mask.state_dict()
  }
  save_checkpoint(state=checkpoint, checkpoint_filename=checkpoint_path, checkpoint_index=epoch)

In [ ]:
# test
best_socre = 0
best_model_index = 0
f = open(train_log_filename, 'a+')
f.write("\n ###Test###")
f.close()

# for i in range(epochs):
for i in range(start, epochs):
  print("\nEpoch:", i)
  load_checkpoint(torch.load(checkpoint_path.replace(".pth.tar", str(i) + ".pth.tar")), model, optimizer_class,optimizer_mask)
  if mode == "incorporate":
    f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_incorporate(loader=test_loader, model=model, device=device)
  if mode == "single":
    f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_single(loader=test_loader, model=model, device=device)
  log_epoch("TEST", i, accuracy_class, f1, dice_class, accuracy_mask, dice_mask, train_log_filename)

  score = f1 + dice_class + dice_mask
  if score > best_socre:
    print("Best is: ", i)
    best_socre = score
    best_model_index = i
    checkpoint = {
      "epoch": i,
      "state_dict": model.state_dict(),
      "optimizer_class": optimizer_class.state_dict(),
      "optimizer_mask": optimizer_mask.state_dict()
    }
    save_checkpoint(state=checkpoint,checkpoint_filename=checkpoint_path)

f = open(train_log_filename, 'a+')
f.write(f"\n Best model is model with index: {best_model_index}")
f.close()

In [ ]:
checkpoint_path = saving_directory + "/checkpoint.pth.tar"
load_checkpoint(torch.load(checkpoint_path), model, optimizer_class,optimizer_mask)
if mode == "incorporate":
  f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_incorporate(loader=test_loader, model=model, device=device)
if mode == "single":
  f1, dice_class, accuracy_class, dice_mask, accuracy_mask = evaluate_model_single(loader=test_loader, model=model, device=device)